In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [0]:
import keras
import tensorflow as tf
import bayesflow.experimental as bf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
@bf.Prior()
def context_prior():
    r = np.random.normal(0.1, 0.01, size=1)
    alpha = np.random.uniform(-0.5 * np.pi, 0.5 * np.pi, size=1)
    
    return {"r": r, "alpha": alpha}

In [ ]:
@bf.Prior()
def prior():
    theta = np.random.uniform(-1.0, 1.0, size=2)
    return {"theta": theta}

In [ ]:
@bf.Likelihood(is_conditional=True)
def simulator(r, alpha, theta):
    x1 = -np.abs(theta[0] + theta[1]) / np.sqrt(2.0) + r * np.cos(alpha) + 0.25
    x2 = (-theta[0] + theta[1]) / np.sqrt(2.0) + r * np.sin(alpha)
    
    return {"x1": x1, "x2": x2}

In [ ]:
generative_model = bf.GenerativeModel(
    context_prior=context_prior,  # TODO: static type?
    prior=prior,
    simulator=simulator
)

In [ ]:
dataset = bf.datasets.OnlineDataset(
    generative_model,
    batch_size=128,
    workers=12,
    use_multiprocessing=True
)

In [ ]:
inference_network = bf.CouplingFlow()

In [ ]:
model = bf.AmortizedPosterior(
    inference_network=inference_network
)

In [ ]:
model.fit(dataset, epochs=100)

In [ ]:
samples = model.sample((128,))

In [ ]:
sns.scatterplot(samples)
plt.show()